# Assignment 1 Group no. []
### Project members: 


### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.


### Instructions
All assignments starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of assignments starting with 
number 2 below, then the assignment will receive 2 points (in total).

It is highly recommended that you do not develop the code directly within the notebook
but that you copy the comments and test cases to your regular development environment
and only when everything works as expected, that you paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above,
and thereby 


## Load NumPy, pandas and time

In [1]:
import numpy as np
import pandas as pd
import time


## Reused functions from Assignment 1

In [2]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
def accuracy(df, correctlabels):
    N = range(len(df))
    pred = [df.iloc[i,:].idxmax() for i in N]
    indicator = [1 if pred[i] == correctlabels[i] else 0 for i in N ]
    accuracy = sum(indicator)/len(indicator)
    return accuracy

def brier_score(df, correctlabels):
    if len(df) != len(correctlabels):
        print("Error in Length")
        return None
    sum_error = 0
    for i in range(len(df)):
        f = df.iloc[i,:]
        idx = np.where(df.columns==correctlabels[i])[0]
        o = [1 if k == idx else 0 for k in range(df.shape[1])]
        sum_error += np.sum((f - o)**2)
    
    return sum_error / len(df)

def binary_auc(pred, true, col):
    
    pred = np.array(pred)
    v = [i == col for i in true] 
    col_pred, other_pred = pred[v], pred[[not i for i in v]]

    predscore = list(pred)
    predscore.extend([0,1])
    predscore = sorted(predscore)
    dictscores = {}

    for thres in predscore:
        if(len(other_pred)==0):
            FPR = 0
        else:
            FPR = sum(other_pred >= thres)/len(other_pred)
        if(len(col_pred)==0):
            TPR = 0
        else:
            TPR = sum(col_pred >= thres)/len(col_pred)
        dictscores[thres] = [FPR, TPR]

    list_reversed = [i for i in reversed(list(dictscores.values()))]

    auc = 0
    if([0,0] not in list_reversed):
        list_reversed = [[0,0]] + list_reversed
    if([1,1] not in list_reversed):
        list_reversed = list_reversed + [[1,1]]
    for i in range(len(list_reversed) - 1):
        tpr_left, tpr_right = list_reversed[i][1], list_reversed[i+1][1]
        fpr_left, fpr_right = list_reversed[i][0], list_reversed[i+1][0]
        if fpr_right != fpr_left:
            height = (tpr_left + tpr_right) / 2
            width = fpr_right - fpr_left
            auc += height*width
    return auc


def auc(df, correctlabels):
    N = len(df)
    auc = 0
    for col in df:
        weight = sum(pd.Series(correctlabels) == col)/N
        bin_auc = binary_auc(df[col], correctlabels, col)
        auc += weight*bin_auc
    return auc

def create_normalization(df, normalizationtype="minmax"):
    new_df = df.copy()
    normalization = {}
    for col in df.columns:
        if col == "CLASS" or col == "ID":
            continue
        if normalizationtype == "minmax":
            a = df[col].min()
            b = df[col].max()
            new_df[col] = (df[col] - a)/(b - a)
            normalization.update({col: (normalizationtype, a, b)})
        elif normalizationtype == "zscore":
            a = df[col].mean()
            b = df[col].std()
            new_df[col] = (df[col] - a)/b
            normalization.update({col: (normalizationtype, a, b)})
        else:
            print("There is no such type!")
        
    return new_df, normalization

def apply_normalization(df, normalization):
    new_df = df.copy()
    for col in normalization:
        normalizationtype = normalization[col][0]
        a = normalization[col][1]
        b = normalization[col][2]
        if normalizationtype == "minmax":
            new_df[col] = (df[col] - a)/(b - a)
            new_df[col] = [x if x < 1 else 1 for x in new_df[col] ] #hint 2
            new_df[col] = [x if x > 0 else 0 for x in new_df[col] ] #hint 2
        elif normalizationtype == "zscore":
            new_df[col] = (df[col] - a)/b
        else:
            print("There is no such type!")
    
    return new_df

def create_imputation(df):
    new_df = df.copy()
    imputation = {}

    for col in df.columns:
        if col not in ["ID", "CLASS"]:
            if df[col].dtype in ['float64', 'float32', 'int64', 'int32']: 
                fillvalue = df[col].mean()
                new_df[col] = new_df[col].fillna(fillvalue)
            elif df[col].dtype == "object" or df[col].dtype.name == "category": 
                fillvalue = df[col].mode()[0]
                new_df[col] = new_df[col].fillna(fillvalue)       
            imputation.update({col: fillvalue})
    return new_df, imputation

def apply_imputation(df, imputation):
    new_df = df.copy()
    for col in df.columns:
        if col not in ['ID','CLASS']:     
            fillvalue = imputation[col]
            new_df[col] = new_df[col].fillna(fillvalue)
    return new_df

def create_bins(df, nobins=10, bintype="equal-width"):
    new_df = df.copy()
    binning = {}
    for col in new_df.columns:
        if col not in ['ID', 'CLASS'] and new_df[col].dtype in ['float64', 'float32', 'int64', 'int32']:
            if bintype == 'equal-width':
                new_df[col], bins = pd.cut(new_df[col], nobins, retbins=True, labels=False)
            elif bintype == 'equal-size':
                new_df[col], bins = pd.qcut(new_df[col], nobins, retbins=True, labels=False, duplicates='drop')
            bins[0], bins[-1] = -np.inf, np.inf
            binning[col] = bins
            new_df[col] = new_df[col].astype('category')
            new_df[col] = new_df[col].cat.set_categories([str(i) for i in new_df[col].cat.categories], rename=True) #int->str
        else:
            new_df[col] = new_df[col].astype('category')
    return new_df, binning


def apply_bins(df, binning):
    new_df = df.copy()
    for col in new_df.columns:
        if(col not in ['ID', 'CLASS'] and new_df[col].dtype in ['float64', 'float32', 'int64', 'int32']):
            bins = binning[col]
            new_df[col] = pd.cut(new_df[col], bins, labels=False)
            new_df[col] = new_df[col].astype('category')
            new_df[col] = new_df[col].cat.set_categories([str(i) for i in new_df[col].cat.categories], rename = True) 
        else:
            new_df[col] = new_df[col].astype('category')
    return new_df

def create_one_hot(df):
    new_df = df.copy()
    one_hot = {}
    for col in df.columns:
        if col not in ['ID', 'CLASS']: 
            if new_df[col].dtype == 'object' or new_df[col].dtype.name ==  'category':
                new_df[col] = new_df[col].astype('category')
                one_hot[col] = new_df[col].cat.categories    
                df_one_hot = pd.get_dummies(new_df[col])
                new_df = new_df.drop(columns=col, axis=1)
                new_df = pd.concat([new_df, df_one_hot], axis=1)
                
    return new_df, one_hot

def apply_one_hot(df, one_hot):
    new_df = df.copy()
    for col in df.columns:
        if col in one_hot.keys():
            for i in one_hot[col]:
                name = col+'_'+i
                new_col = pd.Series((df[col]==i).astype('float'))
                new_df[name] = new_col
            new_df = new_df.drop(columns=col, axis=1)
    return new_df

## 1. Define the class kNN

In [3]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# imputation, normalization, one_hot, labels, training_labels, training_data
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype: "minmax" (default) or "zscore"
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.normalization should be a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot should be a one-hot mapping (see Assignment 1; can be excluded if this function was not completed)
# self.training_labels should be a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels should be the categories of the previous series
# self.training_data should be the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
# normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns 
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self: the object itself
# df: a dataframe
# k: an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are estimated by the relative class frequencies in the set of class labels from the k nearest 
#              (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation, normalization and (possibly) one-hot
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies


In [3]:

class kNN():
    def __init__(self):
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.labels = None
        self.training_labels = None
        self.training_data = None
        
    def fit(self, df, normalizationtype="minmax"):
        self.training_data = df.copy()
        self.training_labels = df["CLASS"].astype("category")
        self.labels = self.training_labels.cat.categories    
        del self.training_data["CLASS"], self.training_data["ID"]
        
        self.training_data, self.imputation = create_imputation(self.training_data)
        self.training_data, self.normalization = create_normalization(self.training_data, normalizationtype=normalizationtype)
        self.training_data, self.one_hot = create_one_hot(self.training_data)
        self.training_data = self.training_data.values

    def get_nearest_neighbor_predictions(self, x_test, k):
        distances = []
        for i in range(len(self.training_data)):
            r = self.training_data[i]
            distances.append(np.linalg.norm(x_test-r))
        dis_df = pd.DataFrame({"Class": self.training_labels, "Dist": distances})
        dis_df = dis_df.sort_values("Dist")
        freq = dis_df[:k]["Class"].value_counts() 
        freq = freq/sum(freq)
        return pd.DataFrame(freq.sort_index()).T
        
    def predict(self, df, k=5):
        self.testing_data = df.copy()
        del self.testing_data["CLASS"]
        if "ID" in self.testing_data.columns:
            del self.testing_data["ID"]
        
        self.testing_data = apply_imputation(self.testing_data, self.imputation)
        self.testing_data = apply_normalization(self.testing_data, self.normalization)
        self.testing_data = apply_one_hot(self.testing_data, self.one_hot)
        self.testing_data = self.testing_data.values
        
        self.predictions = pd.DataFrame()
        for i in range(len(self.testing_data)):
            rt = self.testing_data[i]
            freq = self.get_nearest_neighbor_predictions(rt, k)
            self.predictions = pd.concat([self.predictions,freq])
        self.predictions.reset_index(drop=True, inplace=True)
        return self.predictions

In [4]:
# Test our code 

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

results


Training time: 0.07 s.
Testing time (k=1): 1.37 s.
Testing time (k=3): 1.18 s.
Testing time (k=5): 0.83 s.
Testing time (k=7): 0.89 s.
Testing time (k=9): 0.81 s.


,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.471028,0.833843
7,0.598131,0.471867,0.833481
9,0.616822,0.482981,0.827727


In [5]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.2f}".format(brier_score(predictions,train_labels)))


Accuracy on training set (k=1): 1.00
AUC on training set (k=1): 1.00
Brier score on training set (k=1): 0.00


In [37]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

results


Training time: 0.02 s.
Testing time (k=1): 0.15 s.
Testing time (k=3): 0.14 s.
Testing time (k=5): 0.14 s.
Testing time (k=7): 0.14 s.
Testing time (k=9): 0.14 s.


,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.471028,0.833843
7,0.598131,0.471867,0.833481
9,0.616822,0.482981,0.827727


In [14]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.2f}".format(brier_score(predictions,train_labels)))


Accuracy on training set (k=1): 1.00
AUC on training set (k=1): 1.00
Brier score on training set (k=1): 0.00


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [6]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.class_priors should be a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
# to the relative frequencies of the labels
# self.feature_class_value_counts should be a mapping from a feature (column name) to another mapping, which
# given a feature value and class label provides the number of training instances with this specific combination
# self.feature_class_counts should me a mapping from the feature (column name) and class label to the number of
# training instances with this specific class label and any (non-missing) value for the feature
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: feature_class_value_counts can be a dictionary, which given a feature f returns a mapping obtained 
#         by pandas groupby and size (see lecture slides), which given a feature value v and class label c 
#         returns the number of instances, e.g., using get((c,v),0)
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
# predictions with estimated class probabilities for each row in df, where the class probabilities
# are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply discretization
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors


In [7]:
class NaiveBayes():
    def __init__(self):
        self.binning = None
        self.class_priors = None
        self.feature_class_value_counts = None
        self.feature_class_counts = None

    def fit(self, df, nobins=10, bintype="equal-width"):
        self.training_data = df.copy()
        self.training_labels = df["CLASS"].astype("category")
        self.count = self.training_labels.value_counts()
        self.labels = self.training_labels.cat.categories    
        freq = self.count/sum(self.count)
        self.class_priors = freq.to_dict()
        del self.training_data["ID"]
        
        self.nobins = nobins
        self.bintype = bintype
        
        self.training_data, self.binning = create_bins(self.training_data, nobins=self.nobins, bintype=self.bintype)
        
        
        self.feature_class_value_counts = {} # key: featurename -> (class, feature_values) values: cnt For example:feature_class_value_counts["RI"].get((1,"1"),default=0)
        for col in self.training_data:
            if col != "CLASS":
                dicts = self.training_data.groupby(["CLASS",col])["CLASS"].count().to_dict()
                self.feature_class_value_counts.update({col: dicts})
        
        self.feature_class_counts = {} # key: featurename -> (class)  values: cnt
        for col in self.training_data:
            if col != "CLASS":
                df2 = self.training_data.dropna(axis = 0, how='any', subset = ['CLASS', col])
                dicts = dict(df2.loc[:,'CLASS'].value_counts())
                self.feature_class_counts.update({col: dicts})
        
 
    def predict(self, df):
        self.testing_data = df.copy()
  
        if "CLASS" in self.testing_data.columns:
            del self.testing_data["CLASS"]
        if "ID" in self.testing_data.columns:
            del self.testing_data["ID"]
        
        self.testing_data = apply_bins(self.testing_data, self.binning)
        
        Npts = self.testing_data.shape[0]
        Nclasses = len(self.count) 
        logProb = np.zeros((Nclasses, Npts)) 
        
        # For each test sample and each class, there is a corresponding probability
        
        for i in range(len(self.testing_data)):
            rt = self.testing_data.iloc[i:i+1,:]
            for j in range(len(self.labels)):
                label = self.labels[j]
                
                p_x_yc = [] # P(X|y=label) = P(X1=rt["feature1"]|y=label)*P(X2=rt["feature2"]|y=label)* ...
                for col in rt.columns:
                    xi = rt[col].iloc[0]
                    
                    up = self.feature_class_value_counts[col].get((label, xi), 0)
                    down = self.feature_class_counts[col].get(label, 0) 
                    p_xi_yc = up/down  
                    p_x_yc.append(p_xi_yc)
                
                # prob Here is P(y=label|X=rt) = P(X|y=label)*P(y=label)=data*prior
                prob = self.class_priors[label] 
                for x in p_x_yc:
                    prob *= x
                # logProb[j][i] = np.log(prob) #if needed we could apply np.log to those probabilities
                logProb[j][i] = prob
        
        NormlogProb = logProb.copy()
        
        for j in range(logProb.shape[1]):
            if sum(logProb[:,j]) == 0: # in case this sum is zero
                for k in range(len(self.labels)):
                    NormlogProb[k][j] = self.class_priors[self.labels[k]]
            else:
                for i in range(logProb.shape[0]):
                    NormlogProb[i][j] = logProb[i,j]/sum(logProb[:,j])
        
        predictions = pd.DataFrame(NormlogProb.T)
        predictions.columns = self.labels

        return predictions

In [8]:
# Test our code 

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

results


Training time (3, 'equal-width'): 0.17 s.
Testing time (3, 'equal-width'): 0.36 s.
Training time (3, 'equal-size'): 0.10 s.
Testing time (3, 'equal-size'): 0.36 s.
Training time (5, 'equal-width'): 0.17 s.
Testing time (5, 'equal-width'): 0.36 s.
Training time (5, 'equal-size'): 0.19 s.
Testing time (5, 'equal-size'): 0.35 s.
Training time (10, 'equal-width'): 0.10 s.
Testing time (10, 'equal-width'): 0.36 s.
Training time (10, 'equal-size'): 0.10 s.
Testing time (10, 'equal-size'): 0.34 s.


Accuracy  Brier score       AUC
3  equal-width  0.616822     0.622116  0.724335
   equal-size   0.607477     0.554782  0.780163
5  equal-width  0.644860     0.551101  0.771688
   equal-size   0.598131     0.581556  0.796675
10 equal-width  0.654206     0.527569  0.812887
   equal-size   0.588785     0.741668  0.751165

In [9]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.85
AUC on training set: 0.97
Brier score on training set: 0.23


In [39]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

results


Training time (3, 'equal-width'): 0.09 s.
Testing time (3, 'equal-width'): 0.07 s.
Training time (3, 'equal-size'): 0.06 s.
Testing time (3, 'equal-size'): 0.08 s.
Training time (5, 'equal-width'): 0.07 s.
Testing time (5, 'equal-width'): 0.08 s.
Training time (5, 'equal-size'): 0.08 s.
Testing time (5, 'equal-size'): 0.08 s.
Training time (10, 'equal-width'): 0.15 s.
Testing time (10, 'equal-width'): 0.21 s.
Training time (10, 'equal-size'): 0.14 s.
Testing time (10, 'equal-size'): 0.12 s.


Accuracy  Brier score       AUC
3  equal-width  0.616822     0.622116  0.724335
   equal-size   0.607477     0.554782  0.780163
5  equal-width  0.644860     0.551101  0.771688
   equal-size   0.598131     0.581556  0.796675
10 equal-width  0.654206     0.527569  0.812887
   equal-size   0.588785     0.741668  0.751165

In [40]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.85
AUC on training set: 0.97
Brier score on training set: 0.23


### Comment on assumptions, things that do not work properly, etc.